<a href="https://colab.research.google.com/github/parsakhavarinejad/unet_on_oxford_pet_dataset/blob/main/U_NET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from glob import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, ReduceLROnPlateau

In [2]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
!nvidia-smi

Sun Sep  3 14:03:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
batch_size = 16
lr = 1e-4
epochs = 20
height = 224
width = 224

In [5]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

In [6]:
files_dir = os.path.join("files", "non.aug")
model_file = os.path.join(files_dir, "unet-non-aug.h5")
log_file = os.path.join(files_dir, "log-non-aug.csv")

In [7]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [8]:
create_dir(files_dir)

Conv Block


In [9]:
def conv_block(inputs, num_filters):
  x = Conv2D(num_filters, 3, padding='same')(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters, 3, padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  return x


In [10]:
def encoder_block(inputs, num_filters):
  x = conv_block(inputs, num_filters)
  p = MaxPool2D((2,2))(x)

  return x, p

In [11]:
def decoder_block(inputs, skip, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)

  skip = tf.image.resize(skip, (x.shape[1], x.shape[2]))
  x = Concatenate()([x, skip])
  x = conv_block(x, num_filters)
  return x

In [12]:

def build_unet(input_shape):
  inputs = Input(shape=input_shape)

  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  b1 = conv_block(p4, 1024)

  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  output = Conv2D(1,1, padding='same', activation="sigmoid")(d4)
  model = Model(inputs, output, name='UNET')

  return model


In [13]:
info

tfds.core.DatasetInfo(
    name='oxford_iiit_pet',
    full_name='oxford_iiit_pet/3.2.0',
    description="""
    The Oxford-IIIT pet dataset is a 37 category pet image dataset with roughly 200
    images for each class. The images have large variations in scale, pose and
    lighting. All images have an associated ground truth annotation of breed.
    """,
    homepage='http://www.robots.ox.ac.uk/~vgg/data/pets/',
    data_path='/root/tensorflow_datasets/oxford_iiit_pet/3.2.0',
    file_format=tfrecord,
    download_size=773.52 MiB,
    dataset_size=774.69 MiB,
    features=FeaturesDict({
        'file_name': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=37),
        'segmentation_mask': Image(shape=(None, None, 1), dtype=uint8),
        'species': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    sp

In [14]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

def load_image(datapoint):
  global height, width
  input_image = tf.image.resize(datapoint['image'], (height, width))
  input_mask = tf.image.resize(
    datapoint['segmentation_mask'],
    (height, width),
    method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
  )

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [15]:
train_images = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

In [16]:
class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=42):
    super().__init__()
    # both use the same seed, so they'll make the same random changes.
    self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
    self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)

  def call(self, inputs, labels):
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)
    return inputs, labels

In [17]:
train_batches = (
    train_images
    .cache()
    .shuffle(1000)
    .batch(batch_size)
    .map(Augment())
    .prefetch(buffer_size=tf.data.AUTOTUNE))

test_batches = test_images.batch(batch_size)

In [18]:
# import matplotlib.pyplot as plt
# def display(display_list):
#   plt.figure(figsize=(15, 15))

#   title = ['Input Image', 'True Mask', 'Predicted Mask']

#   for i in range(len(display_list)):
#     plt.subplot(1, len(display_list), i+1)
#     plt.title(title[i])
#     plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
#     plt.axis('off')
#   plt.show()

# for images, masks in train_batches.take(2):
#   sample_image, sample_mask = images[0], masks[0]
#   display([sample_image, sample_mask])

In [19]:
for x, y in train_batches:
  print(x.shape, y.shape)
  break

(16, 224, 224, 3) (16, 224, 224, 1)


In [20]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [21]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

In [22]:
# tf.keras.utils.plot_model(model, show_shapes=True, dpi=200, to_file='model.png')

In [23]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [24]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [25]:
model.fit(train_batches,
          validation_data=test_batches,
          epochs=epochs,
          callbacks=callbacks)

Epoch 1/20
230/230 [==============================] - ETA: 0s - loss: 0.1339 - accuracy: 0.5890
Epoch 1: val_loss improved from inf to 0.38158, saving model to files/non.aug/unet-non-aug.h5
230/230 [==============================] - 256s 911ms/step - loss: 0.1339 - accuracy: 0.5890 - val_loss: 0.3816 - val_accuracy: 0.5772 - lr: 0.0010
Epoch 2/20
230/230 [==============================] - ETA: 0s - loss: -2.6121 - accuracy: 0.6962
Epoch 2: val_loss improved from 0.38158 to -8.49704, saving model to files/non.aug/unet-non-aug.h5
230/230 [==============================] - 241s 1s/step - loss: -2.6121 - accuracy: 0.6962 - val_loss: -8.4970 - val_accuracy: 0.7023 - lr: 0.0010
Epoch 3/20
230/230 [==============================] - ETA: 0s - loss: -8.6932 - accuracy: 0.7434
Epoch 3: val_loss improved from -8.49704 to -9.67929, saving model to files/non.aug/unet-non-aug.h5
230/230 [==============================] - 240s 1s/step - loss: -8.6932 - accuracy: 0.7434 - val_loss: -9.6793 - val_accur

KeyboardInterrupt: ignored